In [ ]:
!pip install requests
!pip install python_dotenv

In [37]:
import requests
import os
from dotenv import load_dotenv

file_path = os.path.abspath('')
dotenv_path=os.path.join(file_path, '..', '.env')
print(f'.env file  path: {dotenv_path}')
load_dotenv(dotenv_path)
PORT = os.getenv('port')
DATABASE_URL = os.getenv('DATABASE_URL')
print(f'Port: {PORT}')

.env file  path: a:\projects\chinook\notebooks\..\.env
Port: 8001


# Test Case 1
Iterate over all the "all" endpoints and print the results we expect to see success and items here

In [38]:
# Define the API endpoint and parameters
endpoints = ['albums', 'tracks', 'playlists', 'genres', 'mediatypes', 
             'artists', 'customers', 'employees', 'invoices', 'invoiceitems', 
             'playlisttracks']
for endpoint in endpoints:
    url = f'http://127.0.0.1:{PORT}/api/v1/{endpoint}/'
    print(url)
    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        print("Success:", response.json())
    else:
        print("Failed:", response.status_code)

http://127.0.0.1:8001/api/v1/albums/
Success: [{'id': 2, 'title': 'Balls to the Wall', 'artist': None, 'artist_id': 2}, {'id': 3, 'title': 'Restless and Wild', 'artist': None, 'artist_id': 2}, {'id': 4, 'title': 'Let There Be Rock', 'artist': None, 'artist_id': 1}, {'id': 5, 'title': 'Big Ones', 'artist': None, 'artist_id': 3}, {'id': 6, 'title': 'Jagged Little Pill', 'artist': None, 'artist_id': 4}, {'id': 7, 'title': 'Facelift', 'artist': None, 'artist_id': 5}, {'id': 8, 'title': 'Warner 25 Anos', 'artist': None, 'artist_id': 6}, {'id': 9, 'title': 'Plays Metallica By Four Cellos', 'artist': None, 'artist_id': 7}, {'id': 10, 'title': 'Audioslave', 'artist': None, 'artist_id': 8}, {'id': 11, 'title': 'Out Of Exile', 'artist': None, 'artist_id': 8}, {'id': 12, 'title': 'BackBeat Soundtrack', 'artist': None, 'artist_id': 9}, {'id': 13, 'title': 'The Best Of Billy Cobham', 'artist': None, 'artist_id': 10}, {'id': 14, 'title': 'Alcohol Fueled Brewtality Live! [Disc 1]', 'artist': None, 'a

# Test Case 2
We select the first item so id = 1. 
Again we expect to see success and some items here

In [39]:
for endpoint in endpoints:
    url = f'http://127.0.0.1:{PORT}/api/v1/{endpoint}/1'
    print(url)
    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        print("Success:", response.json())
    else:
        print("Failed:", response.status_code)

# playlisttracks has none for 1 so get one it has
url = f'http://127.0.0.1:{PORT}/api/v1/playlisttracks/61006'
print(url)
response = requests.get(url)
if response.status_code == 200:
    print("playlisttracks Success:", response.json())
else:
    print("playlisttracks Failed:", response.status_code)

http://127.0.0.1:8001/api/v1/albums/1
Success: {'id': 1, 'title': 'For Those About To Rock We Salute You', 'artist': None, 'artist_id': 1}
http://127.0.0.1:8001/api/v1/tracks/1
Success: {'id': 1, 'name': 'For Those About To Rock (We Salute You)', 'album_id': 1, 'media_type_id': 1, 'genre_id': 1, 'composer': 'Angus Young, Malcolm Young, Brian Johnson', 'milliseconds': 343719, 'bytes': 11170334, 'unit_price': 0.99, 'playlistTrack': None}
http://127.0.0.1:8001/api/v1/playlists/1
Success: {'id': 1, 'name': 'Music', 'playlistTrack': None}
http://127.0.0.1:8001/api/v1/genres/1
Success: {'id': 1, 'name': 'Rock'}
http://127.0.0.1:8001/api/v1/mediatypes/1
Success: {'id': 1, 'name': 'MPEG audio file'}
http://127.0.0.1:8001/api/v1/artists/1
Success: {'id': 1, 'name': 'AC/DC', 'album': None}
http://127.0.0.1:8001/api/v1/customers/1
Success: {'id': 1, 'first_name': 'Luís', 'last_name': 'Gonçalves', 'company': 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'address': 'Av. Brigadeiro Faria Lima,

# Test Case 3
We select the first item in sthe list of all items so id: 1. 
We store that values.
We then put some new data to the endpoint (using name for example).
We then select that same item again and compare the updated values to see if they have been updated.
We then revert the data to the original values and check again.

# Test Case 3.1 
for 'tracks', 'playlists', 'genres', 'mediatypes', 'artists', 'playlists'
update and reset the name field.

In [40]:
def check_update_key(endpoint, key, value):
    print(f'check_update_key: {endpoint} {key} {value}')
    # Make the GET request
    data = requests.get(endpoint).json()
    original_val = data[key]
    data[key] = value
    # Make the PUT request with the updated value
    response = requests.put(endpoint, json=data)
    #print(f'Response from put: {response.json()}')
    updated_data = requests.get(endpoint).json()
    if updated_data[key] == value:
        print(f'Success: we updated the [{key} -> {updated_data[key]}] json: {updated_data}')
    else:
        print("Failed:", response.status_code)
    # Reset the value
    updated_data[key] = original_val
    response = requests.put(endpoint, json=updated_data)
    #print(f'Response from put: {response.json()}')
    reset_data = requests.get(endpoint).json()
    if original_val == reset_data[key]:
        print(f'Success: we reset the [{key} -> {reset_data[key]}] json: {reset_data}')
    else:
        print("Failed:", response.status_code)


In [41]:
# these items all have "name" fields so it makes it easy to test as a group
# ideally we would test each json key for update
withnames = ['tracks', 'playlists', 'genres', 'mediatypes', 'artists', 'playlists']
for endpoint in withnames:
    endpoint = f'http://127.0.0.1:{PORT}/api/v1/{endpoint}/1'
    check_update_key(endpoint, 'name', 'test_00001')

check_update_key: http://127.0.0.1:8001/api/v1/tracks/1 name test_00001
Success: we updated the [name -> test_00001] json: {'id': 1, 'name': 'test_00001', 'album_id': 1, 'media_type_id': 1, 'genre_id': 1, 'composer': 'Angus Young, Malcolm Young, Brian Johnson', 'milliseconds': 343719, 'bytes': 11170334, 'unit_price': 0.99, 'playlistTrack': None}
Success: we reset the [name -> For Those About To Rock (We Salute You)] json: {'id': 1, 'name': 'For Those About To Rock (We Salute You)', 'album_id': 1, 'media_type_id': 1, 'genre_id': 1, 'composer': 'Angus Young, Malcolm Young, Brian Johnson', 'milliseconds': 343719, 'bytes': 11170334, 'unit_price': 0.99, 'playlistTrack': None}
check_update_key: http://127.0.0.1:8001/api/v1/playlists/1 name test_00001
Success: we updated the [name -> test_00001] json: {'id': 1, 'name': 'test_00001', 'playlistTrack': None}
Success: we reset the [name -> Music] json: {'id': 1, 'name': 'Music', 'playlistTrack': None}
check_update_key: http://127.0.0.1:8001/api/v

# Test Case 3.2 
for 'album' update and reset the title field.
for customer and employees update and reset the firstname field
for invoices update and reset the total field
for invoice_items update and reset the quantity field

In [42]:
albums_endpoint = f'http://127.0.0.1:{PORT}/api/v1/albums/1'
check_update_key(albums_endpoint, 'title', 'test_00001')

customers_endpoint = f'http://127.0.0.1:{PORT}/api/v1/customers/1'
check_update_key(customers_endpoint, 'first_name', 'test_00001')

employees_endpoint = f'http://127.0.0.1:{PORT}/api/v1/employees/1'
check_update_key(employees_endpoint, 'first_name', 'test_00001')

invoices_endpoint = f'http://127.0.0.1:{PORT}/api/v1/invoices/1'
check_update_key(invoices_endpoint, 'total', 999999.99)

invoiceitems_endpoint = f'http://127.0.0.1:{PORT}/api/v1/invoiceitems/1'
check_update_key(invoiceitems_endpoint, 'quantity', 9999)

# here I need to select a track that exixts and update it with a track id that also exists
playlisttracks_endpoint = f'http://127.0.0.1:{PORT}/api/v1/playlisttracks/69721'
check_update_key(playlisttracks_endpoint, 'track_id', 3389)


check_update_key: http://127.0.0.1:8001/api/v1/albums/1 title test_00001
Success: we updated the [title -> test_00001] json: {'id': 1, 'title': 'test_00001', 'artist': None, 'artist_id': 1}
Success: we reset the [title -> For Those About To Rock We Salute You] json: {'id': 1, 'title': 'For Those About To Rock We Salute You', 'artist': None, 'artist_id': 1}
check_update_key: http://127.0.0.1:8001/api/v1/customers/1 first_name test_00001
Success: we updated the [first_name -> test_00001] json: {'id': 1, 'first_name': 'test_00001', 'last_name': 'Gonçalves', 'company': 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'address': 'Av. Brigadeiro Faria Lima, 2170', 'city': 'São José dos Campos', 'state': 'SP', 'country': 'Brazil', 'postal_code': '12227-000', 'phone': '+55 (12) 3923-5555', 'fax': '+55 (12) 3923-5566', 'email': 'luisg@embraer.com.br', 'support_rep_id': 3, 'invoice': None}
Success: we reset the [first_name -> Luís] json: {'id': 1, 'first_name': 'Luís', 'last_name': 'Gonçalves

In [58]:
def post_and_delete(endpoint):
    print(f'post_and_delete: {endpoint}')
    # Make the GET request
    data = requests.get(endpoint).json()[1]
    del data["id"] # remove the id field we will be posting this
    print(f'First item data in list: {data}')
    response = requests.post(endpoint, json=data)
    print(f'Response from post: {response.status_code} {response.json()}')
    if response.status_code == 200:
        print(f'Success: we posted data [{data} -> {response}')
    else:
        print("Failed:", response.status_code)
    id = response.json()['id']
    delete_endpoint = f'{endpoint}{id}'
    print(f'Delete endpoint: {delete_endpoint}')
    response = requests.delete(delete_endpoint)
    print(f'Response from delete: {response.status_code} {response.json()}')
    if response.status_code == 200:
        print(f'Success: we delted data [{data} -> {response}')
    else:
        print("Failed:", response.status_code)


In [59]:
endpoints = ['albums', 'tracks', 'playlists', 'genres', 'mediatypes', 
             'artists', 'customers', 'employees', 'invoices', 'invoiceitems', 
             'playlisttracks']
for endpoint in endpoints:
    url = f'http://127.0.0.1:{PORT}/api/v1/{endpoint}/'
    post_and_delete(url)


post_and_delete: http://127.0.0.1:8001/api/v1/albums/
First item data in list: {'title': 'Restless and Wild', 'artist': None, 'artist_id': 2}
Response from post: 200 {'id': 354, 'title': 'Restless and Wild', 'artist': None, 'artist_id': 2}
Success: we posted data [{'title': 'Restless and Wild', 'artist': None, 'artist_id': 2} -> <Response [200]>
Delete endpoint: http://127.0.0.1:8001/api/v1/albums/354
Response from delete: 200 {'id': 354, 'title': 'Restless and Wild', 'artist': None, 'artist_id': 2}
Success: we delted data [{'title': 'Restless and Wild', 'artist': None, 'artist_id': 2} -> <Response [200]>
post_and_delete: http://127.0.0.1:8001/api/v1/tracks/
First item data in list: {'name': 'Fast As a Shark', 'album_id': 3, 'media_type_id': 2, 'genre_id': 1, 'composer': 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 'milliseconds': 230619, 'bytes': 3990994, 'unit_price': 0.99, 'playlistTrack': None}
Response from post: 200 {'id': 3506, 'name': 'Fast As a Shark', 'album_id': 3,